In [1]:
import os
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [25]:
df = pd.read_csv("../static_merged_data.csv")
x = df.drop(labels="status", axis=1)
y = df[["status"]]

def prepare_train_test_set(X, Y, num_classes):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, shuffle=True, test_size=.2)
    train_labels = y_train.status.astype('category').cat.codes.to_numpy()
    test_labels = y_test.status.astype('category').cat.codes.to_numpy()
    X_train.to_numpy()
    X_test.to_numpy()
    return (X_train, X_test), (train_labels, test_labels)

In [26]:
(X_train, X_test), (train_labels, test_labels) = prepare_train_test_set(x,y,3)

In [28]:
from keras.models import Model
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import Input, InputLayer
import tensorflow as tf
from keras import Sequential
from keras.layers import Activation 
import tensorflow_model_optimization as tfmot
model = Sequential([
    InputLayer(input_shape=(4,), name="input_layer"),
    tfmot.quantization.keras.quantize_annotate_layer(Dense(64, activation="relu")),
    tfmot.quantization.keras.quantize_annotate_layer(Dense(64, activation="relu")),
    tfmot.quantization.keras.quantize_annotate_layer(Dense(128, activation="relu")),
    tfmot.quantization.keras.quantize_annotate_layer(Dense(256, activation="relu")),
    tfmot.quantization.keras.quantize_annotate_layer(Dense(128, activation="relu")),
    tfmot.quantization.keras.quantize_annotate_layer(Dense(16, activation="relu")),
    Dense(3, name="output_layer")
], name="best_quant_model")

model.summary()

Model: "best_quant_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_annotate_27 (Quant  (None, 64)               320       
 izeAnnotate)                                                    
                                                                 
 quantize_annotate_28 (Quant  (None, 64)               4160      
 izeAnnotate)                                                    
                                                                 
 quantize_annotate_29 (Quant  (None, 128)              8320      
 izeAnnotate)                                                    
                                                                 
 quantize_annotate_30 (Quant  (None, 256)              33024     
 izeAnnotate)                                                    
                                                                 
 quantize_annotate_31 (Quant  (None, 128)         

In [29]:
from keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from keras.optimizers import Adam

# Use `quantize_apply` to actually make the model quantization aware.
quant_aware_model = tfmot.quantization.keras.quantize_apply(model)

quant_aware_model.summary()

model.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])

# model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath=(model.name + ".h5"),
#     save_weights_only=False,
#     monitor='val_acc',
#     mode='max',
#     save_best_only=True)

history = model.fit(X_train, train_labels, epochs=80, validation_split=0.1)

Epoch 1/80
2168/2168 [==============================] - 6s 3ms/step - loss: 1.1004 - accuracy: 0.3397 - val_loss: 1.0982 - val_accuracy: 0.3335
Epoch 2/80
2168/2168 [==============================] - 5s 2ms/step - loss: 1.0982 - accuracy: 0.3411 - val_loss: 1.0980 - val_accuracy: 0.3335
Epoch 3/80
2168/2168 [==============================] - 5s 2ms/step - loss: 1.0981 - accuracy: 0.3422 - val_loss: 1.0981 - val_accuracy: 0.3335
Epoch 4/80
2168/2168 [==============================] - 5s 2ms/step - loss: 1.0981 - accuracy: 0.3421 - val_loss: 1.0980 - val_accuracy: 0.3502
Epoch 5/80
2168/2168 [==============================] - 5s 2ms/step - loss: 1.0981 - accuracy: 0.3408 - val_loss: 1.0981 - val_accuracy: 0.3335
Epoch 6/80
2168/2168 [==============================] - 5s 2ms/step - loss: 1.0981 - accuracy: 0.3395 - val_loss: 1.0978 - val_accuracy: 0.3502
Epoch 7/80
1741/2168 [=======================>......] - ETA: 0s - loss: 1.0981 - accuracy: 0.3423

KeyboardInterrupt: 

In [23]:
import tensorflow_model_optimization as tfmot
quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for quantization aware
q_aware_model = quantize_model(model)

# 'quantized_model' requires a recompile
q_aware_model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

q_aware_model.summary()

Model: "best_quant_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_2 (QuantizeL  (None, 4)                3         
 ayer)                                                           
                                                                 
 quant_dense (QuantizeWrappe  (None, 64)               325       
 rV2)                                                            
                                                                 
 quant_dense_1 (QuantizeWrap  (None, 64)               4165      
 perV2)                                                          
                                                                 
 quant_dense_2 (QuantizeWrap  (None, 128)              8325      
 perV2)                                                          
                                                                 
 quant_dense_3 (QuantizeWrap  (None, 256)         

In [24]:
train_subset = X_train[0:1000]      # out of 60000
train_labels_subset = train_labels[0:1000]

q_aware_model.fit(train_subset, train_labels_subset,batch_size=512, epochs=1, validation_split=0.1)

2/2 [==============================] - 3s 313ms/step - loss: 3.7730 - accuracy: 0.3500 - val_loss: 3.4456 - val_accuracy: 0.3300


In [21]:
_, baseline_model_accuracy = model.evaluate(X_test,test_labels, verbose=0)
_, q_aware_model_accuracy = q_aware_model.evaluate(X_test,test_labels,verbose=0)

print("Baseline test accuracy: ", baseline_model_accuracy)
print("Quantized test accuracy: ", q_aware_model_accuracy)

Baseline test accuracy:  0.9539179801940918
Quantized test accuracy:  0.33933576941490173
